<a href="https://colab.research.google.com/github/MK316/Engpro/blob/main/Engpro23_Step01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 💛 Data (23. 6. 3)

+ Engpro Spring 23 (N=26, F=21, M=5)
+ **Rainbow passage**

## Data process
+ foramt to .wav
+ stereo to mono
+ heading removed
+ saved in Google asrdata as a zip: Engpro23PPre.zip

# [1] Info file to read and data summary
+ File from Github
+ Engpro23_info.csv: 26 files with info

|Index|Subject|Gender
|--|--|--|
|1|F01|F|

In [ ]:
import pandas as pd

url = "https://github.com/MK316/Engpro/raw/main/data/Engpro23_info.csv"
df = pd.read_csv(url)
df.tail()

# Rainbow passage
[Github link](https://raw.githubusercontent.com/MK316/Engpro/main/data/rainbow.txt)

In [ ]:
import requests
url = "https://raw.githubusercontent.com/MK316/Engpro/main/data/rainbow.txt"
response = requests.get(url)
rainbow = response.text
rainbow


In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/Engpro/main/data/rainbow_sents.csv"
sents = pd.read_csv(url)
sents.head()

# [2] Reading audio files from Google Drive (zip file)

## [2A] Audio files from Google to Colab 'myaudio' directory

In [ ]:

#@markdown 🎯Mount Google drive and list files (e.g., "asrdata" folder in my case)
from google.colab import drive 
import os

drive.mount('/content/drive')

mydir = input("Type the file directory in your google drive: e.g., asrdata  ")
!ls "/content/drive/MyDrive/{mydir}"
!pwd
     

In [ ]:
#@markdown 🎯 To do  

#@markdown   [1] Make a new folder: type a new folder name (e.g., myaudio)
import os

folder_name = input("Type a name for the new folder.")

if not os.path.exists(folder_name):
  os.makedirs(folder_name)
  print(f"A new folder name '{folder_name}' has been created.")
else:
  print(f"{folder_name} already exists.")

#@markdown [2] Unzip files: type a zip file name (e.g., SE.zip)

zipname = input("Type your zip file name (e.g., se.zip) to process (unzip and save them under the new folder")
!unzip "/content/drive/MyDrive/asrdata/{zipname}" -d "/content/{folder_name}/"

print(f"Your {zipname} is unzipped under '{folder_name}' folder")

In [ ]:
#@markdown 🎯 Unmount Google drive (clearing): optional

from google.colab import drive
drive.flush_and_unmount()

## Filelist

In [ ]:
import os

fname = []
path = '/content/myaudio/'

# use listdir() function to list files
files = os.listdir(path)

for file in files:
  fname.append(file)

filename = fname.sort()
print(filename)


In [ ]:
fname

In [ ]:
df_sorted = df.sort_values(by="Subject")
print(df_sorted)
df_sorted['Filename'] = fname
df_sorted

In [ ]:
df_sorted.to_csv('Engpro23PPRE_info.csv',index=False)

# File from github: Engpro23PPRE_info.csv

In [ ]:
url = "https://raw.githubusercontent.com/MK316/Engpro/main/data/Engpro23PPRE_info.csv"
df = pd.read_csv(url)
df

## [2B] Openai

In [ ]:
!pip install openai

In [ ]:
import openai
openai.api_key = input('openai API key here')

### Reading audio from colab

In [ ]:
#@markdown 🎯Install {autotime} to measure runtime (From here, runtime appears automatically.)
%%capture
!pip install ipython-autotime
%load_ext autotime

In [ ]:
# A single file process to check
# file = df['Filename'][0]
audio_file = open('/content/myaudio/PF01.wav', "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file,language="en-US")
print(transcript['text'])

In [ ]:
os.chdir('/content/myaudio')

recog = []

for i in range(0, len(df['Subject'])):
  file = df['Filename'][i]
  audio_file = open(file, "rb")
  transcript = openai.Audio.transcribe("whisper-1", audio_file,language="en")
  recog.append(transcript['text'])
print(recog)

In [ ]:
df['Recognized'] = recog
df.tail()

In [ ]:
# Comparison between target and recognized
se = df

In [ ]:
#@markdown Calculate recognition rate, record missing words => dataframe (df2) 
from nltk.tokenize import RegexpTokenizer

# number of words in the sentence
nws = []
# number of words in the recognized text
nwr = []
# number of missing words 
nmw = []
# number of words actually recognized
nwar = []

# Recgonition Rate
rr = []
#
nr = []
# Missing word list
mw = []
# Correctly recognized word list
recword=[]


for i in range(0,len(se['Subject'])):
  t1 = rainbow

# text 1
  txt1 = t1.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist = tokenizer.tokenize(txt1)
  nt = len(wlist)
  nws.append(nt)

# text 2
  t2 = se['Recognized'][i]
  txt2 = t2.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist1 = tokenizer.tokenize(txt2)
  nt1 = len(wlist1)
  nwr.append(nt1)

# Recognition rate

  # from tables.idxutils import calc_chunksize
  # from nltk.downloader import ErrorMessage
# txt1(original text), txt2 (recognized text)
 
  mword = []
  rword = []
  score = 0
  for i in range(0, len(wlist1)):
      w = wlist1[i]

      if w in wlist:
        sc = 1
        rword.append(w)
      else:
        sc = 0
        mword.append(w)

      score = score + sc
      mwords = ', '.join(mword)
      rwords = ', '.join(rword)
  mw.append(mwords)
  missingword = round((score/len(wlist))*100,2)
  nr.append(score)
  recword.append(rwords)

  # RecRate = float(format(missingword, '.2f'))
  # ErrRate = float(format((100.0 - RecRate), '.2f'))
  
  rr.append(missingword)
  # print('Matching words: %d'%score, 'out of %d words'%len(wlist))
  # print("="*50)
  # print('Recognition Rate: %f %%'%RecRate)
  # print('Error Rate: %f %%'%ErrRate)

se['LenS'] = nws
se['LenR'] = nwr
se['N_RecW'] = nr
se['RecRate'] = rr
se['Recognized_words'] = recword
se['MissRec_words'] = mw

se.head(); se.tail()

In [ ]:
se.describe()

In [ ]:
df1 = se
df1.to_csv("EngproPPRE_recognized.csv", index=False)

# 💜 From here: df (with recognized words and texts)

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/Engpro/main/data/EngproPPRE_recognized.csv"
df = pd.read_csv(url)
df.head()

In [ ]:
# data = df[['Filename', 'N_RecW','RecRate']]
target = []

for i in range(0, len(df['Filename'])):
  target.append(rainbow)
df['Target']=target
df.tail()

# WER

In [ ]:
#@markdown + Install and import libraries
%%capture
!pip install Levenshtein
import Levenshtein as lev
import numpy as np
import pandas as pd

In [ ]:
#@markdown + Calculate WER and WER_lev and add columns to df
# import numpy as np
# import pandas as pd
# import Levenshtein as lev
#@markdown + Column names 'Original' and 'Recognized' for comparison
def calculate_wer_word_level(s1, s2):
    """
    Calculation of WER with Dynamic Time Warping algorithm at word level.
    """

    # splitting the sentences into words
    s1_words = s1.split()
    s2_words = s2.split()

    # preparing the matrix for the Dynamic Time Warping algorithm
    dtw_matrix = np.zeros((len(s1_words)+1, len(s2_words)+1))
    for i in range(len(s1_words)+1):
        for j in range(len(s2_words)+1):
            if i == 0:
                dtw_matrix[0][j] = j
            elif j == 0:
                dtw_matrix[i][0] = i

    # Dynamic Time Warping algorithm
    for i in range(1, len(s1_words)+1):
        for j in range(1, len(s2_words)+1):
            cost = 0 if s1_words[i-1] == s2_words[j-1] else 1
            dtw_matrix[i][j] = cost + min([dtw_matrix[i-1][j], dtw_matrix[i][j-1], dtw_matrix[i-1][j-1]])

    wer = dtw_matrix[len(s1_words)][len(s2_words)] / len(s1_words)

    return wer

def calculate_wer_lev(s1, s2):
    """
    Calculation of WER with Levenshtein distance at character level.
    """

    # splitting the sentences into words
    s1_words = s1.split()
    s2_words = s2.split()

    # calculating Levenshtein distance
    edit_distance = lev.distance(' '.join(s1_words), ' '.join(s2_words))

    # normalizing by the length of the original sentence (s1)
    wer = edit_distance / len(s1_words)

    return wer


In [ ]:
# Assuming df is your DataFrame and it has columns 'Sentence' and 'Recognized'
import numpy as np

wer_values_word_level = df.apply(lambda row: calculate_wer_word_level(row['Target'], row['Recognized']), axis=1)
wer_values_lev = df.apply(lambda row: calculate_wer_lev(row['Target'], row['Recognized']), axis=1)

# Adding WER values as new columns to the DataFrame
df['WER'] = wer_values_word_level
df['WER_lev'] = wer_values_lev

In [ ]:
df.to_csv('Engpro23PPRE_result.csv')

# Result preview

In [ ]:
os.chdir('/content')

In [ ]:
url = "https://raw.githubusercontent.com/MK316/Engpro/main/data/Engpro23PPRE_result.csv"
df = pd.read_csv(url)
df.describe()


MissRec_words

In [ ]:
n_missing = []

for i in range(0,len(df['Filename'])):
  wlist = df['MissRec_words'][i]
  w = wlist.split(',')
  n = len(w)
  n_missing.append(n)

df['N_missing'] = n_missing
df.describe()

# 1) Regression: Missing words ~ WER

In [ ]:
import statsmodels.formula.api as smf

# Assuming df is your dataframe and it has columns 'LOR' and 'WER'
# Create a fitted model
model = smf.ols(formula='WER ~ N_missing', data=df).fit()

# Print out the statistics of the model
print(model.summary())


Let's break down the results:

R-squared: This is the coefficient of determination. It explains the proportion of the variance in the dependent variable that is predictable from the independent variable. In this case, 15.3% of the variability in WER can be explained by LOR. This isn't particularly high, which might suggest that other variables not included in the model could also be impacting WER.

Adj. R-squared: This is the R-squared value that has been adjusted based on the number of predictors in the model. It is more accurate and should be used over the regular R-squared if you have more than one predictor. Since this model has only one predictor (LOR), the adjusted R-squared is very close to the regular R-squared.

coef: These are the coefficients of the linear regression equation. The equation would look something like this: WER = 0.0473 + 0.0072*LOR. That means, for each unit increase in LOR, WER increases by 0.0072 units, on average, assuming all other variables are held constant.

P>|t|: This is the p-value associated with the null hypothesis that the coefficient is equal to zero (no effect). A p-value of less than 0.05 is typically considered statistically significant at the 5% level. In this case, the p-value for LOR is 0.020, which is less than 0.05, suggesting that LOR has a statistically significant effect on WER.

[0.025 0.975]: These are the 95% confidence intervals for the coefficients. This means we are 95% confident that the true population coefficient for LOR lies between 0.001 and 0.013.

Durbin-Watson: This tests for the presence of autocorrelation (a relationship between values separated from each other by a given time lag) in the residuals. Values range from 0 to 4, and values around 2 suggest no autocorrelation. Your value of 1.890 suggests that there is no significant autocorrelation.

Prob (F-statistic): This is the p-value of the overall regression model. It tests the null hypothesis that all regression coefficients are equal to zero. A p-value of less than 0.05 indicates that at least one of the predictors is significantly related to the outcome variable. In this case, the p-value is 0.0201, which is less than 0.05, indicating that the model is statistically significant.

Finally, it's important to remember that correlation does not imply causation. While LOR might be associated with WER, it doesn't necessarily mean that changes in LOR cause changes in WER.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Regression line plot
sns.regplot(x='N_missing', y='WER', data=df)
plt.show()

# Residuals plot
residuals = df['WER'] - model.predict()
sns.residplot(x=model.predict(), y=residuals, lowess=True, color="g")
plt.show()

# QQ plot
import statsmodels.api as sm
sm.qqplot(residuals, line ='45')
plt.show()

# Histogram of residuals
plt.hist(residuals)
plt.show()

# Correlation matrix
corrMatrix = df.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()


There are a few ways to visually summarize the results of your regression analysis:

Scatter plot with fitted line (Regression line plot): You can create a scatter plot of the data and add the line of best fit from your model. This will provide a visualization of how well the line fits the data. You can also plot the residuals (the differences between the observed and predicted values) to see how evenly they're distributed around the line.

Residuals plot: A residuals plot shows the difference between the observed and predicted values (the residuals) for each observation, typically plotted against the predicted values. If your model is a good fit, the points in the residuals plot should be randomly and evenly dispersed around zero.

QQ Plot: A Quantile-Quantile plot, or QQ plot, is used to check if your data follows a particular distribution. In the case of linear regression, we often want to check if the residuals follow a normal distribution. If the points roughly fall along the diagonal line, then the residuals are normally distributed.

Histogram of residuals: This can also be used to check if residuals are normally distributed.

Correlation matrix: While not directly related to the regression results, a correlation matrix can give you an overview of how all your variables relate to each other.

Here is how you can plot some of these in Python:

# 2) Boxplot

In [ ]:
df1['Category'] = pd.qcut(df1['WER'], 4, labels=['A', 'B', 'C', 'D'])
df1.head()

# df1.to_csv('Engpro23PPRE_results_total.csv', index=False)

In [ ]:
bins = [0, 0.1, 0.2, 0.3, float('inf')]
labels = ['A', 'B', 'C', 'D']

df1['WER_cat'] = pd.cut(df1['WER'], bins=bins, labels=labels)


In [ ]:
df1.to_csv('EngproPPRE_resultstotal.csv', index=False)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a boxplot
sns.boxplot(data=df[['N_missing']])
plt.show()


## Excluding outlier

In [ ]:
df1 = df[df['N_missing'] <= 30]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a boxplot
sns.boxplot(data=df1[['WER']])
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and a set of subplots
fig, axs = plt.subplots(1, 2, figsize=(10,5))

# First boxplot
axs[0].boxplot(df1['N_missing'])
axs[0].set_title('Boxplot of Number of missing owrds')
axs[0].set_ylim([0,25])  # Set y limits for first plot

# Second boxplot
axs[1].boxplot(df1['WER'])
axs[1].set_title('Boxplot of WER')
axs[1].set_ylim([0,0.25])  # Set y limits for second plot

plt.tight_layout()
plt.show()


In [ ]:
summary = df1.groupby(['Category', 'WER_cat']).size().reset_index(name='Count')
print(summary)

In [ ]:
pivot_table = df1.pivot_table(index='Category', columns='WER_cat', aggfunc='size', fill_value=0)
print(pivot_table)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.boxplot(x='Category', y='WER', data=df1)

plt.title('Boxplot of WER by Category')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.boxplot(x='WER_cat', y='WER', data=df1)

plt.title('Boxplot of WER by Category')
plt.show()


### Different scales: N_missing and WER

In [ ]:
import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()

# Plot the first data series in blue on the left y-axis
ax1.set_ylabel('First Data Series', color='b')
ax1.plot(df['N_missing'], color='b')
ax1.tick_params(axis='y', labelcolor='b')

# Then create a second y-axis for the second data series
ax2 = ax1.twinx()

# Plot the second data series in red on the right y-axis
ax2.set_ylabel('Second Data Series', color='r')
ax2.plot(df['WER'], color='r')
ax2.tick_params(axis='y', labelcolor='r')

fig.tight_layout()
plt.show()
